# Module 5 - Programming Assignment

## Directions

There are general instructions on Blackboard and in the Syllabus for Programming Assignments. This Notebook also has instructions specific to this assignment. Read all the instructions carefully and make sure you understand them. Please ask questions on the discussion boards or email me at `EN605.445@gmail.com` if you do not understand something.

<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
You must follow the directions *exactly* or you will get a 0 on the assignment.
</div>

You must submit a zip file of your assignment and associated files (if there are any) to Blackboard. The zip file will be named after you JHED ID: `<jhed_id>.zip`. It will not include any other information. Inside this zip file should be the following directory structure:

```
<jhed_id>
    |
    +--module-05-programming.ipynb
    +--module-05-programming.html
    +--(any other files)
```

For example, do not name  your directory `programming_assignment_01` and do not name your directory `smith122_pr1` or any else. It must be only your JHED ID.

In [1]:
from IPython.core.display import *
from StringIO import StringIO
from random import random, gauss, randrange
import bisect, sys

## Local Search - Genetic Algorithm

For this assignment we're going to use the Genetic Algorithm to find the solution to a shifted Sphere Function in 10 dimensions, $x$, where the range of $x_i$ in each dimension is (-5.12 to 5.12). Here a "solution" means the vector $x$ that minimizes the function. The Sphere Function is:

$$f(x)=\sum x^2_i$$

We are going to shift it over 0.5 in every dimension:

$$f(x) = \sum (x_i - 0.5)^2$$

where $n = 10$.

As this *is* a minimization problem you'll need to use the trick described in the lecture to turn the shifted Sphere Function into an appropriate fitness function (which is always looking for a *maximum* value).

## Binary GA

You are going to solve the problem two different ways. First, using the traditional (or "Canonical") Genetic Algorithm that encodes numeric values as binary strings (you don't have to represent them literally as strings but they are general lists or strings of only 0 or 1).

There are many different ways to affect this encoding. For this assignment, the easiest is probably to use a 10 bit binary encoding for each $x_i$. This gives each $x_i$ a potential value of 0 to 1024 which can be mapped to (-5.12, 5.12) by subtracting 512 and dividing by 100.

All the GA operators will be as described in the lecture.

**Important**

Please remember that there is a difference between the *genotype* and the *phenotype*. The GA operates on the *genotype* (the encoding) and does not respect the boundaries of the phenotype (the decoding). So, for example, do **not** use a List of Lists to represent an individual. It should be a *single* List of 10 x 10 or 100 bits. In general, crossover and mutation have no idea what those bits represent.

## Real Valued GA

For the real valued GA, you can represent each $x_i$ as a float in the range (-5.12, 5.12) but you will need to create a new mutation operator that applies gaussian noise. Python's random number generator for the normal distribution is called `gauss` and is found in the random module:

```
from random import gauss, random
```

You may need to experiment a bit with the standard deviation of the noise but the mean will be 0.0.

## GA

The Genetic Algorithm itself will have the same basic structure in each case: create a population, evaluate it, select parents, apply crossover and mutation, repeat until the number of desired generations have been generated. The easiest way to accomplish this in "Functional" Python would be to use Higher Order Functions.



Your code should print out the best individual of each generation including the generation number, genotype (the representation), phenotype (the actual value), the fitness (based on your fitness function transformation) and the function value (for the shifted sphere) if passed a DEBUG=True flag.

The GA has a lot of parameters: mutation rate, crossover rate, population size, dimensions (given for this problem), number of generations.  You can put all of those and your fitness function in a `Dict` in which case you need to implement:

```python
def binary_ga( parameters):
  pass
```

and

```python
def real_ga( parameters):
  pass
```

Remember that you need to transform the sphere function into a legit fitness function. Since you also need the sphere function, I would suggest that your parameters Dict includes something like:

```python
parameters = {
   "f": lambda xs: sphere( 0.5, xs),
   "minimization": True
   # put other parameters in here.
}
```

and then have your code check for "minimization" and create an entry for "fitness" that is appropriate.

In [2]:
def sphere( shift, xs):
    return sum( [(x - shift)**2 for x in xs])

In [3]:
sphere( 0.5, [1.0, 2.0, -3.4, 5.0, -1.2, 3.23, 2.87, -4.23, 3.82, -4.61])

113.42720000000001


-----

### Common Helper Functions ###

** crossover(parents1, parents2) **

** cumsum(L) **

** pickParents(fitness, nParents) **

In [4]:
def crossover(parents1, parents2):
    pt = randrange(len(parents1))
    return parents1[:pt] + parents2[pt:]

def cumsum(L):
    total = 0
    for l in L:
        total += l
        yield total
        
def pickParents(fitness, nParents):
    avgFit = sum(fitness) / len(fitness) # avg fitness
    tmp = [(n,f) for n,f in enumerate(fitness) if f>avgFit]
    if len(tmp) > 5:
        idx,eligible = zip(*tmp)
    else:
        idx,eligible = range(len(fitness)),fitness
    cumFit = list(cumsum( eligible ))
    rands = [random()*cumFit[-1] for n in xrange(nParents)]
    return [idx[bisect.bisect_left(cumFit,r)] for r in rands]

def fitFn(pm, genes):
    return pm['maxVal'] - pm['f']( pm['pheno'](genes) )

def updateBest(pop, fitness, bestGene, bestFit, pm):
    iterBestIdx = fitness.index(max(fitness))
    if fitness[iterBestIdx] > bestFit:
        bestGene = pop[iterBestIdx]
        bestFit = fitness[iterBestIdx]
    return bestGene, bestFit, pop[iterBestIdx], fitness[iterBestIdx]

def printGenInfo(genNum, gene, fit, pm):
    out = 'Gen %s best gene: %s\n' + \
        '\tx-values: %s \n\tfitness: %.4f, sphere value: %.4f\n'
    pheno = pm['repr'](gene)
    xVal = pm['pheno'](gene)
    xRepr = '[' + ', '.join('%.3g' % f for f in xVal)[:-2] + ']'
    sp = pm['f'](xVal)
    print out % (str(genNum), str(pheno), str(xRepr), fit, sp)

### Genetic Algorithm  Main Program ###

In [ ]:
def geneticAlgo(pm, DEBUG=False):
    pop = pm['initFn'](pm)
    
    bestGene,bestFit = None,-sys.maxsize # initialize best 
    for gen in xrange(pm['nGeneration']):
        fitness = [fitFn(pm, genes) for genes in pop]
        bestGene,bestFit,g,f = updateBest(pop, fitness, bestGene, bestFit, pm)
        
        if DEBUG and gen%pm['printEvery']==0:
            printGenInfo(gen, g, f, pm)
        idx = pickParents(fitness, pm['popSize']*2)
        nextGen = list()
        for n in xrange(pm['popSize']):
            child = crossover(pop[idx[n]], pop[idx[-n]])
            if random() < pm['mutateProb']:
                pm['mutateFn'](child, pm)
            nextGen.append(child)
        pop = nextGen
    
    bestGene,bestFit,g,f = updateBest(pop, fitness, bestGene, bestFit, pm)
    printGenInfo('final', bestGene, bestFit, pm)
    return pm['pheno'](bestGene)

-----

## Binary GA ##

### Binary GA Helper Functions###

In [5]:
def getDecimals(genes):
    bins = [genes[n:n+10] for n in xrange(0,len(genes),10)]
    return [(int(x,2)-511)/100.0 for x in bins]

def mutateBinary(genes, pm):
    i = randrange(len(genes))
    genes = genes[:i] + ('0' if genes[i]=='1' else '1') + genes[i+1:]
    return genes

def initBinGA(pm):
    tmp = [[format(randrange(2**pm['nBits']-1), '#012b')[2:] \
            for x in xrange(10)] for y in xrange(pm['popSize'])]
    return [''.join(p) for p in tmp]

### Binary GA Parameters and Run ###

In [7]:
## Traditional GA

param = {
    "f": lambda xs: sphere( 0.5, xs),
    "popSize": 1000,
    "mutateProb": 0.05,
    "nGeneration": 100,
    "nBits": 10,
    "maxVal": sphere(0.5, [-5.12 for x in xrange(10)]),
    "pheno" : lambda x: getDecimals(x),
    "initFn": initBinGA, 
    "mutateFn": mutateBinary,
    "repr": lambda x: [x[n:n+10] for n in xrange(0,len(x),10)],
    "printEvery": 10
}

bestX =  geneticAlgo( param, True)

Gen 0 best gene: ['0101100010', '1011111110', '1000111001', '0111000111', '0111101110', '1000110111', '0011100101', '1010001010', '0111110111', '0111010010']
	x-values: [-1.57, 2.55, 0.58, -0.56, -0.17, 0.56, -2.82, 1.39, -0.08, -0.] 
	fitness: 292.7207, sphere value: 23.1233

Gen 10 best gene: ['1001001011', '1000100101', '1000111101', '0110110111', '1000011011', '0111100011', '1000011011', '1000111000', '1001001100', '1000110011']
	x-values: [0.76, 0.38, 0.62, -0.72, 0.28, -0.28, 0.28, 0.57, 0.77, 0.] 
	fitness: 313.4758, sphere value: 2.3682

Gen 20 best gene: ['1001000000', '1000110000', '1001000100', '1000100011', '1001100010', '1000000101', '1000011011', '0111110011', '1001001100', '1000111000']
	x-values: [0.65, 0.49, 0.69, 0.36, 0.99, 0.06, 0.28, -0.12, 0.77, 0.] 
	fitness: 314.8214, sphere value: 1.0226

Gen 30 best gene: ['1001000011', '1000100101', '1000100110', '1001100010', '1000100010', '1000111101', '1000110111', '1000101011', '1000010111', '1000110000']
	x-values: [0.68

---

## Real Value GA ##

### Real Value GA Helper Functions ###

In [9]:
def mutateReal(genes, p):
    i = randrange(len(genes))
    genes[i] = min( max(genes[i]*gauss(0,p['sigma']),-p['xMax']) , p['xMax'])
    return genes

def initRealGA(pm):
    return [[random()*2*pm['xMax'] - pm['xMax'] for x in xrange(10)] \
           for y in xrange(pm['popSize'])]

In [10]:
## Real Valued GA

param2 = {
    "f": lambda xs: sphere( 0.5, xs),
    "popSize": 1000,
    "mutateProb": 0.01,
    "nGeneration": 100,
    "nBits": 10,
    "maxVal": sphere(0.5, [-5.12 for x in xrange(10)]),
    "sigma": 0.5,
    "xMax": 5.12,
    
    "pheno" : lambda x: x,
    "initFn": initRealGA, 
    "mutateFn": mutateReal,
    "repr": lambda x: '[' + ', '.join('%.3f' % f for f in x)[:-2] + ']',
    "printEvery": 10
    
}

bestXreal = geneticAlgo( param2, False)

Gen final best gene: [0.443, 0.473, 0.501, 0.519, 0.461, 0.512, 0.550, 0.435, 0.502, 0.5]
	x-values: [0.443, 0.473, 0.501, 0.519, 0.461, 0.512, 0.55, 0.435, 0.502, 0.5] 
	fitness: 315.8309, sphere value: 0.0131

